In [10]:
import requests
import json
from os.path import expanduser
from requests.auth import HTTPBasicAuth

with open(expanduser('brain_credentials.txt')) as f:
    credentials = json.load(f)

username, password = credentials

sess = requests.Session()

sess.auth = HTTPBasicAuth(username,password)

response = sess.post('https://api.worldquantbrain.com/authentication')

print(response.status_code)
print(response.json())

201
{'user': {'id': 'TX41606'}, 'token': {'expiry': 14400.0}, 'permissions': []}


In [ ]:
def get_datafields(s,searchScope,dataset_id:str='',search: str=''):
    import pandas as pd 
    instrument_type = searchScope['instrumentType']
    region = searchScope['region']
    delay = searchScope['delay']
    universe = searchScope['universe']
    if len(search) == 0:
        url_template = "https://api.worldquantbrain.com/data-fields?" + \
            f"&instrmentType={instrument_type}" + \
            f"&region={region}&delay={str(delay)}&universe={universe}&dataset.id={dataset_id}&limit=50" +\
            "&offset={x}"
        count = s.get(url_template.format(x=0)).json()['count']
    else:
        url_template = "https://api.worldquantbrain.com/data-fields?" + \
            f"&instrmentType={instrument_type}" + \
            f"&region={region}&delay={str(delay)}&universe={universe}&limit=50" +\
            f"&search={search}" +\
            "&offset={x}"
        count = 100
    datafields_list = []
    for x in range(0,count,50):
        datafields = s.get(url_template.format(x=x))
        datafields_list.append(datafields.json()['results'])
    datafields_list_flat = [item for sublist in datafields_list for item in sublist]
    datafields_df = pd.DataFrame(datafields_list_flat)
    return datafields_df

In [ ]:
searchScope = {'region': 'USA','delay': '1','universe':'TOP3000','instrumentType':'EQUITTY'}
model38 = get_datafields(s=sess,searchScope=searchScope,dataset_id='model38')
datafields_list_model38 = model38['id'].values

In [ ]:
alpha_list = []
for datafield in datafields_list_model38:
    print("正在将如下Alpha表达式和setting封装")
    print('group_rank(ts_rank({datafiled},252,industry))')
    simulation_data = {
        'type' : 'REGULAR',
        'settings': {
            'instrumentType': 'EQUITY',
            'region': 'USA',
            'universe': 'TOP3000',
            'delay': 1,
            'decay': 0,
            'neutralization': 'INDUSTRY',
            'truncation': 0.08,
            'pasteurization': 'ON',
            'unitHandling': 'VERIFY',
            'nanHandling': 'OFF',
            'language': 'FASTEXPR',
            'visualization': False
        },
        'regular': f'group_rank(ts_rank({datafield},252),subindustry)'
    }

    alpha_list.append(simulation_data)

In [ ]:
from time import sleep

for alpha in alpha_list:
    sim_resp = sess.post(
        'https://api.worldquantbrain.com/simulations',
        json=alpha,
    )
    #print(sim_resp.headers)
    try:
        sim_process_url = sim_resp.headers["Location"]

        while True:
            sim_process_resp = sess.get(sim_process_url)
            retry_after_sec = float(sim_process_resp.headers.get("Retry-After",0))
            if retry_after_sec == 0:
                break
            sleep(retry_after_sec)
        alpha_id = sim_process_resp.json()["alpha"]
        print(alpha_id)
    except:
        print("no location,sleep for 5 seconds and try next alpha")
        sleep(10)

In [16]:
simulation_data  = {
    'type' : 'REGULAR',
    'settings': {
        'instrumentType': 'EQUITY',
        'region': 'USA',
        'universe': 'TOP3000',
        'delay': 1,
        'decay': 0,
        'neutralization': 'INDUSTRY',
        'truncation': 0.08,
        'pasteurization': 'ON',
        'unitHandling': 'VERIFY',
        'nanHandling': 'OFF',
        'language': 'FASTEXPR',
        'visualization': False
    },
    'regular': 'liabilities/assets'
}

In [19]:
from time import sleep

sim_resp = sess.post(
    'https://api.worldquantbrain.com/simulations',
    json=simulation_data,
)
#print(sim_resp.headers)
sim_process_url = sim_resp.headers["Location"]

while True:
    sim_process_resp = sess.get(sim_process_url)
    retry_after_sec = float(sim_process_resp.headers.get("Retry-After",0))
    if retry_after_sec == 0:
        break
    sleep(retry_after_sec)
alpha_id = sim_process_resp.json()["alpha"]
print(alpha_id)

R8vW1Ld
